In [18]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv(r"C:\Users\sebas\OneDrive\Documentos\Datafalcons\df-sales-pbi-main\sales-data\Amazon Sale Report.csv")

C:\Users\sebas\AppData\Local\Temp\ipykernel_15492\1168129.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\sebas\OneDrive\Documentos\Datafalcons\df-sales-pbi-main\sales-data\Amazon Sale Report.csv")


In [15]:
df.columns

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22'],
      dtype='object')

In [16]:
df['Amount_EUR'] = df['Amount']*0.011

In [20]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Crear una copia del dataframe original
df_expanded = df.copy()

# Duplicar datos hacia atrás hasta 2020
for años_atras in range(1, 3):  # 1 año atrás (2021), 2 años atrás (2020)
    nuevo_df = df.copy()
    nuevo_df['Date'] = nuevo_df['Date'] - pd.DateOffset(years=años_atras)

    # Reducir 'Qty' en 1 o 2 unidades si es mayor a 0
    nuevo_df['Qty'] = nuevo_df['Qty'].apply(lambda x: max(0, x - np.random.randint(1, 3)) if x > 0 else x)

    # Reducir 'Amount' y 'Amount_EUR' en un 10-15%
    factor_reduccion = np.random.uniform(0.85, 0.90)  # Reduce entre 10-15%
    for col in ['Amount', 'Amount_EUR']:
        if col in nuevo_df.columns:
            nuevo_df[col] = nuevo_df[col] * factor_reduccion

    # Agregar los datos generados al dataset expandido
    df_expanded = pd.concat([df_expanded, nuevo_df], ignore_index=True)

# Guardar el nuevo dataset con datos hasta 2020
df_expanded.to_excel("dataset_expandido_hasta_2020.xlsx", index=False)

Agregar metricas que faltaban 
@NICO falta hacer que estas metricas afecten al df con los datos historicos

In [6]:
def hash_asin(asin):
    return abs(hash(asin)) % 11  # Genera un valor entre 0 y 10

# 📌 Calcular COGS Unitario (35% - 45% del precio unitario)
df["COGS Unit"] = ((df["Amount_EUR"] / df["Qty"]) * (0.35 + (df["ASIN"].apply(hash_asin) / 100))).round(2)

# 📌 Calcular COGS Total
df["COGS"] = df["COGS Unit"] * df["Qty"]

# 📌 Calcular Other Expenses (15% - 25% del Amount_EUR)
df["Other Expenses"] = (df["Amount_EUR"] * (0.15 + (df["ASIN"].apply(hash_asin) / 100))).round(2)

# 📌 Calcular Gross Profit
df["Gross Profit"] = df["Amount_EUR"] - df["COGS"]

# 📌 Calcular Net Profit
df["Net Profit"] = df["Gross Profit"] - df["Other Expenses"]

In [7]:
df.columns

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22', 'Amount_EUR', 'COGS Unit', 'COGS',
       'Other Expenses', 'Gross Profit', 'Net Profit'],
      dtype='object')

In [9]:
df.describe()

C:\Users\sebas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,index,Qty,Amount,ship-postal-code,Amount_EUR,COGS Unit,COGS,Other Expenses,Gross Profit,Net Profit
count,128975.000000,128975.000000,121180.000000,128942.000000,121180.000000,121180.00,116044.000000,121180.000000,116044.000000,116044.000000
mean,64487.000000,0.904431,648.561465,463966.236509,7.134176,inf,2.856858,1.424781,4.290669,2.863369
std,37232.019822,0.313354,281.211687,191476.764941,3.093329,NaN,1.257323,0.655352,1.890913,1.353204
min,0.000000,0.000000,0.000000,110001.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
25%,32243.500000,1.000000,449.000000,382421.000000,4.939000,1.98,1.960000,0.960000,2.925000,1.914000
50%,64487.000000,1.000000,605.000000,500033.000000,6.655000,2.74,2.680000,1.310000,4.019000,2.615000
75%,96730.500000,1.000000,788.000000,600024.000000,8.668000,3.64,3.510000,1.770000,5.255000,3.604000
max,128974.000000,15.000000,5584.000000,989898.000000,61.424000,inf,23.550000,11.480000,38.704000,28.264000


In [10]:
df.drop(columns=["index", "Unnamed: 22"], inplace=True)

In [11]:
df['Amount_EUR'].max()

np.float64(61.424)

### Agregador de Data Historica

In [12]:
min_date = df['Date'].min()
max_date = df['Date'].max()
print(f"Rango de fechas: {min_date} - {max_date}")

Rango de fechas: 03-31-22 - 06-29-22


### Descargar Archivo

In [35]:
df.to_csv("amazon-sale-report-clean.csv", index=False)